In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from helper_functions import *

N_SAMPLES = 10000
NUM_CLASSES = 10
NUM_FEATURES = 10
RANDOM_SEED = 42

print(f"Torch: version {torch.__version__}")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
!nvidia-smi

In [ ]:
# Multiclass Classification

In [ ]:
# Create the dataset
X_blob, y_blob = make_blobs(n_samples=N_SAMPLES,
                            n_features=NUM_FEATURES,
                            centers=NUM_CLASSES,
                            cluster_std=.2,
                            random_state=RANDOM_SEED)

In [ ]:
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.float)

In [ ]:
X_blob, y_blob

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(X_blob[:, 0], X_blob[:, 1], c = y_blob)

In [ ]:
# Lets build the model
class MultiClass(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(2, 4)        

    def forward(self, x : torch.tensor):
        self.l1(x)

model = MultiClass()


In [ ]:
def printClass(X_, y_, n_classes):
    y_train_blob = torch.zeros(len(y_))
    for i in range(len(y_train_blob)):
        for c in range(n_classes):
            if int(y_[i][c].item()) == 1:
                y_train_blob[i] = c

    plt.figure(figsize=(10, 7))
    plt.scatter(X_[:, 0], X_[:, 1], c = y_train_blob)

# Calculate accuracy - out of 100 what percent the model gets right
def accuracy_fn(y_test, preds):
    correct = (y_test == preds).type(torch.float)
    nc = (torch.count_nonzero(correct))
    return  nc.item() / len(y_test) * 100

y = torch.zeros(N_SAMPLES, NUM_CLASSES)
for i in range(len(y_blob)):
    y[i][int(y_blob[i].item())] = 1

# Model with non linearity
model = nn.Sequential(
    nn.Linear(NUM_FEATURES, 8),
    nn.ReLU(),
    nn.Linear(8, 8),
    nn.Linear(8, NUM_CLASSES),
)

X_train, X_test, y_train, y_test = train_test_split(X_blob, y, test_size=0.33, random_state=432)
X_train, X_test, y_train, y_test

printClass(X_train, y_train, NUM_CLASSES)

# Training
# Loss for classification
loss_fn = nn.MultiLabelSoftMarginLoss() # This is also the output activation function; so is combining Sigmoid -> BCELoss 
# and is more numerically stable

optimizer = optim.SGD(
    params=model.parameters(),
    lr = 0.03
)
# Training
torch.manual_seed(RANDOM_SEED)
epochs = 5000
for ep in range(epochs):
    model.train()

    # Logits
    y_logits = model(X_train).squeeze()
    # Using sigmoid activation and rounding
    y_pred = torch.round(torch.sigmoid(y_logits))
    # loss and accuracy
    loss = loss_fn(y_logits, # we expect raw logits
                   y_train)

    optimizer.zero_grad()

    # backpropagation
    loss.backward()

    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        # test loss
        test_loss = loss_fn(test_logits, y_test)

    if ep % (epochs / 10) == 0:
        print(f"Ep: {ep} | loss: {loss:.5f} | tloss: {test_loss:.5f} ")

# Lets visualize
with torch.inference_mode():
    test_logits = model(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
printClass(X_test, test_pred, NUM_CLASSES)


In [ ]:
# hello